## Load data

In [1]:
import os
import glob
import numpy as np
from tensorflow.keras import layers
from tensorflow import keras 
import tensorflow as tf

all_files = ['airplane', 'ant', 'axe', 'banana', 'bicycle', 'bridge', 'butterfly', 'castle', 'cat', 'chair', 'diamond', 'donut', 'The_Eiffel_Tower']
i = 0
while i < len(all_files):
    all_files[i] = "full_numpy_bitmap_" + all_files[i] + ".npy"
    i +=1 

max_items_per_class = 10000

x = np.empty([0, 784])
y = np.empty([0])
class_names = []

#load a subset of the data to memory 
for idx, file in enumerate(all_files):
    data = np.load("Images/" + file)
    data = data[0: max_items_per_class, :]
    labels = np.full(data.shape[0], idx)
    x = np.concatenate((x, data), axis=0)
    y = np.append(y, labels)
    class_name, ext = os.path.splitext(os.path.basename(file))
    class_names.append(class_name.replace('full_numpy_bitmap_', ''))

#separate into training and testing 
permutation = np.random.permutation(y.shape[0])
x = x[permutation, :]
y = y[permutation]

vfold_size = int(x.shape[0]/100*(0.2*100))

x_test = x[0:vfold_size, :]
y_test = y[0:vfold_size]

x_train = x[vfold_size:x.shape[0], :]
y_train = y[vfold_size:y.shape[0]]

2024-04-20 23:50:09.583075: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-20 23:50:09.660081: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-20 23:50:10.014159: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-20 23:50:10.014326: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-20 23:50:10.063744: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

## Preprocess the data

In [2]:
# Reshape and normalize
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1).astype('float32')
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1).astype('float32')

x_train /= 255.0
x_test /= 255.0

# Convert class vectors to class matrices
y_train = keras.utils.to_categorical(y_train, len(class_names))
y_test = keras.utils.to_categorical(y_test, len(class_names))

## Build the model

In [3]:
# Define model
model = keras.Sequential()
model.add(layers.Convolution2D(16, (3, 3),
                        padding='same',
                        input_shape=x_train.shape[1:], activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Convolution2D(32, (3, 3), padding='same', activation= 'relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Convolution2D(64, (3, 3), padding='same', activation= 'relu'))
model.add(layers.MaxPooling2D(pool_size =(2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(len(class_names), activation='softmax')) 
# Train model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['top_k_categorical_accuracy'])
model.summary()

2024-04-20 23:50:30.409264: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2024-04-20 23:50:30.409365: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:129] retrieving CUDA diagnostic information for host: 183sawyer
2024-04-20 23:50:30.409376: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:136] hostname: 183sawyer
2024-04-20 23:50:30.409571: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:159] libcuda reported version is: 525.147.5
2024-04-20 23:50:30.409636: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:163] kernel reported version is: 525.147.5
2024-04-20 23:50:30.409645: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:241] kernel version seems to match DSO: 525.147.5


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 16)        160       
                                                                 
 max_pooling2d (MaxPooling2  (None, 14, 14, 16)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 7, 7, 32)          0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 7, 7, 64)          18496     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 3, 3, 64)          0

In [4]:
#fit the model 
model.fit(x_train, y_train, batch_size = 256, verbose=2, epochs=10)

#evaluate on unseeFor prediction we use model.predict this will return probabilities of the shape [N, 100n data
score = model.evaluate(x_test, y_test, verbose=0)
print('Test accuracy: {:0.2f}%'.format(score[1] * 100))

Epoch 1/10
407/407 - 29s - loss: 0.6939 - top_k_categorical_accuracy: 0.9554 - 29s/epoch - 72ms/step
Epoch 2/10
407/407 - 24s - loss: 0.3413 - top_k_categorical_accuracy: 0.9826 - 24s/epoch - 60ms/step
Epoch 3/10
407/407 - 25s - loss: 0.2809 - top_k_categorical_accuracy: 0.9853 - 25s/epoch - 61ms/step
Epoch 4/10
407/407 - 24s - loss: 0.2492 - top_k_categorical_accuracy: 0.9868 - 24s/epoch - 59ms/step
Epoch 5/10
407/407 - 24s - loss: 0.2262 - top_k_categorical_accuracy: 0.9883 - 24s/epoch - 59ms/step
Epoch 6/10
407/407 - 24s - loss: 0.2062 - top_k_categorical_accuracy: 0.9898 - 24s/epoch - 59ms/step
Epoch 7/10
407/407 - 24s - loss: 0.1900 - top_k_categorical_accuracy: 0.9906 - 24s/epoch - 58ms/step
Epoch 8/10
407/407 - 24s - loss: 0.1763 - top_k_categorical_accuracy: 0.9912 - 24s/epoch - 58ms/step
Epoch 9/10
407/407 - 24s - loss: 0.1645 - top_k_categorical_accuracy: 0.9919 - 24s/epoch - 59ms/step
Epoch 10/10
407/407 - 24s - loss: 0.1524 - top_k_categorical_accuracy: 0.9929 - 24s/epoch -

In [6]:
model.save('pretrained_model.h5')